In [1]:
# Train api
# pytorch 직접구현

In [2]:
# 전처리
from transformers import BertTokenizer
BERT_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_NAME)

In [3]:
# 데이터 준비
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [4]:
nltk.download('movie_reviews') 
ids = movie_reviews.fileids()
reviews = [movie_reviews.raw(id_) for id_ in ids]
categories = [movie_reviews.categories(id_)[0] for id_ in ids]

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\31799\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [5]:
# 라벨인코딩
y = [1 if category == 'pos' else 0 for category in categories]

In [6]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    reviews, y, test_size=0.2, random_state=42
)

In [7]:
# 데ㅣ터셋
import torch
from torch.utils.data import Dataset, DataLoader
class MovieReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            key:val[idx].clone().detach() for key, val in self.encodings.items()
        }
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
# 토큰화 (BertTokenizerFast)
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(BERT_NAME)
train_encodings = tokenizer(
    X_train,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)
test_encodings = tokenizer(
    X_test,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)
train_dataset = MovieReviewDataset(train_encodings, y_train)
test_dataset = MovieReviewDataset(test_encodings, y_test)

In [16]:
# Train API
%pip install -q 'accelerate>=0.26.0'

from transformers import Trainer, TrainingArguments, BertForSequenceClassification
from sklearn.metrics import classification_report
import evaluate
model = BertForSequenceClassification.from_pretrained(
    BERT_NAME,
    num_labels=2
)
accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')
def compute_metrics(eval_pred):
    logits, labels =  eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'],
        'precision': precision_metric.compute(predictions=predictions, references=labels, average='weighted')['precision'],
        'recall': recall_metric.compute(predictions=predictions, references=labels, average='weighted')['recall'],
        'f1': f1_metric.compute(predictions=predictions, references=labels, average='weighted')['f1'],
    }

# TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to='none'    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: "'accelerate": Expected package name at the start of dependency specifier
    'accelerate
    ^


Note: you may need to restart the kernel to use updated packages.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`